In [2]:
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(11,8)})
sns.set(font_scale=1.2)
import numpy as np
from utils import fix_wrong_record_id, fill_id, format_datetime_columns, delete_redundant_columns, mark_inattentive_participants, fix_age,remove_test_id, delete_negative_age
from merge_datasets import split_to_intersection_and_union, fix_wrong_groups, resolve_contradiction
from columns_index_for_merging import columns_dict, merging_functions_by_columns_set, weird_text_columns

In [3]:
def impute_from_column(df, impute_to, impute_from):
    """
    test:
    df[(df['c_ssrs_6'].isna()) & (~df['c_ssrs_last_visit_6'].isna())][['c_ssrs_6', 'c_ssrs_last_visit_6']]
    df = impute_from_column(df, impute_to = 'c_ssrs_6', impute_from = 'c_ssrs_last_visit_6')
    df[(df['c_ssrs_6'].isna()) & (~df['c_ssrs_last_visit_6'].isna())][['c_ssrs_6', 'c_ssrs_last_visit_6']]
    
    """
    df[impute_to] = np.where(df[impute_to].isnull(), df[impute_from], df[impute_to])
    
    return df

In [4]:
df_2022 = pd.read_csv(r'../preprocessed_data/2022_data_imputed_with_clin_stu.csv', na_values= ' ')
df_2021 = pd.read_csv(r'../preprocessed_data/2021_data_imputed_with_clin_stu.csv', na_values= ' ')

df_2022 = delete_negative_age(df_2022)
df_2021 = delete_negative_age(df_2021)

df_2021 = impute_from_column(df_2021, impute_to = 'age_child_pre', impute_from = 'age_child_pre_first')

C:\Users\nogur\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (44,51,53,99,325,366,439,453,459,464,466,501,507,512,514,521,530,532,569,571,580,599,766,771,777,781,786,788,793,795,804,806,826,843,845,854,873,883,918,941,979,991,1002,1040,1045,1047,1049,1051,1053,1054,1080,1096,1119,1122,1126,1127,1131,1132,1133,1134,1136,1137,1138,1141,1142,1144,1161,1183,1186,1237,1239,1259,1261,1262,1263,1269,1270,1271,1272,1384,1387,1389,1405,1408,1411,1415,1432,1434,1435,1436,1442,1443,1444,1445,1499,1568,1569,1572,1573) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\nogur\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (48,1109,1166,1171,1344,1347,1563,1564,1565,1567,1586) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# the preparing datasets

In [5]:
df_2021 = fix_wrong_record_id(df_2021, year=2021)
df_2022 = fix_wrong_record_id(df_2022, year=2022)

##### fill id function - only for 2022

In [6]:
df_2022 = fill_id(df_2022)

In [7]:
df_2021 = format_datetime_columns(df_2021)
df_2022 = format_datetime_columns(df_2022)

In [8]:
df_2021 = mark_inattentive_participants(df_2021, year=2021)
df_2022 = mark_inattentive_participants(df_2022, year=2022)

In [9]:
df_2021, df_2022 = delete_redundant_columns(df_2021 = df_2021, df_2022 = df_2022)

# merging the datasets

In [10]:
union, intersection_2021, intersection_2022 = split_to_intersection_and_union(df_2021 = df_2021, df_2022=df_2022)

In [11]:
union.shape

(1615, 1547)

## handling contradictions

In [12]:
intersection_df = pd.merge(intersection_2021, intersection_2022, on=['id', 'redcap_event_name'], how='outer', suffixes=('_2021', '_2022'))

In [13]:
intersection_df.shape

(344, 3053)

In [14]:
# create intersection_columns_list.csv file
#pd.DataFrame(intersection_2021.columns).to_csv('intersection_columns_list.csv', index=False)

In [ ]:
result = intersection_df.copy()

for key, value in merging_functions_by_columns_set.items():
    print(key, value)
    columns = columns_dict[key]
    for col in columns:
        result[col] = result.apply(resolve_contradiction, args = (f'{col}_2021', f'{col}_2022', value, col), axis=1)
        result = result.drop([f'{col}_2021', f'{col}_2022'], axis=1)

## iterate over the other columns
        
# validate
# i = columns.iloc[0]
# result2[(~result2[f'{i}_2022'].isna()) & (~result2[f'{i}_2021'].isna())][[i, f'{i}_2021', f'{i}_2022']]

visit_date_stu <built-in function max>
diagnosis_binary <built-in function max>
complaint_binary <built-in function max>
timestamp <built-in function min>
date <built-in function min>
groups <built-in function max>
age_child_pre <built-in function max>
record_id take 2021
chameleon_binary_0_1 <built-in function max>
chameleon_binary_1_2 <built-in function min>
opening_psychothe_binary_0_1 <built-in function max>
opening_psychothe_binary_1_2 <built-in function min>
opening_medicine_binary_0_1 <built-in function max>
opening_medicine_binary_1_2 <built-in function min>
with_who_m <built-in function max>
with_who_f <built-in function max>
duplicated_cols take 2022


In [ ]:
intersection = fix_wrong_groups(result)
intersection['original_dataset'] = 'both 2021 & 2022'

In [ ]:
# dd = pd.DataFrame(intersection.columns)
# dd[dd[0].duplicated()].to_csv('duplicated_cols.csv')

In [ ]:
merged_df = pd.concat([union, intersection])

In [ ]:
merged_df = remove_test_id(merged_df)
merged_df = fill_missing_age(merged_df)

In [19]:
#[i for i in merged_df.columns if 'age' in i.lower()]


In [20]:
merged_df.query("age_child_pre < 0")[["age_child", "age_child_pre", "id"]]

,age_child,age_child_pre,id
281,NaN,-99.0,N2178
382,NaN,-99.0,9920
383,NaN,-99.0,9920
388,NaN,-99.0,LH003
389,NaN,-99.0,LH003
390,NaN,-99.0,LH003
391,NaN,-99.0,LH003
600,NaN,-99.0,R6441
979,NaN,-99.0,M8511
986,NaN,-99.0,O8253


In [24]:
(merged_df.groupby('id').age_child.count() == 0).sum()

392

In [25]:
(merged_df.groupby('id').age_child_pre.count() == 0).sum()

26

In [20]:
merged_df.to_csv(r'../preprocessed_data/merged_2021_and_2022.csv', index=False)